## Задание 1
Построить алгоритмы классификации для решения задачи классификации MNIST. 
1. Обучить модели (SVM, logisitic regression, decision tree, (дополнительные модели по желанию) на исходных данных;
2. Использовать алгоритмы PCA и t-SNE для снижения размерности. Какое количество компонент необходимо, чтобы описать 90% дисперсии в PCA?
3. Обучить те же модели на сниженном признаковом пространстве, Какой алгоритм снижения дает лучшие метрики точности моделей классификации? Есть ли улучшение относительно работы моделей на исходном признаковом пространстве?
4. Оцените, какое количество времени требуется для препроцессинга PCA и t-SNE.

In [1]:
from timeit import default_timer as timer

from sklearn.datasets import load_digits
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split

In [2]:
random_state = 12
log_reg_max_iter = 10000

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [3]:
# пайплайны без предобработки
basic_pipelines = [
    ("SVC", Pipeline([("clf", SVC(random_state=random_state))])),
    ("LogReg", Pipeline([("clf", LogisticRegression(random_state=random_state, max_iter=log_reg_max_iter))])),
    ("DecTree", Pipeline([("clf", DecisionTreeClassifier(random_state=random_state))])),
]

basic_predicts = []

start = timer()

for name, pipeline in basic_pipelines:
    pipeline.fit(X_train, y_train)
    predict = pipeline.predict(X_test)
    # сразу посчитаем метрики
    print(f"{name} accuracy: {accuracy_score(y_true=y_test, y_pred=predict)}")
    basic_predicts.append((name, predict))

end = timer()
print(f"Time elapsed: {end - start}")
    

SVC accuracy: 0.9866666666666667
LogReg accuracy: 0.9688888888888889
DecTree accuracy: 0.8666666666666667
Time elapsed: 2.02589975


In [4]:
# используем понижение размерности (PCA)

def pca():
    return PCA(random_state=random_state, n_components=0.90)

pca_pipelines = [
    ("SVC", Pipeline([("pca", pca()), ("clf", SVC(random_state=random_state))])),
    ("LogReg", Pipeline([("pca", pca()), ("clf", LogisticRegression(random_state=random_state, max_iter=log_reg_max_iter))])),
    ("DecTree", Pipeline([("pca", pca()), ("clf", DecisionTreeClassifier(random_state=random_state))])),
]

start = timer()
for name, pipeline in pca_pipelines:
    pipeline.fit(X_train, y_train)
    predict = pipeline.predict(X_test)
    # сразу посчитаем метрики
    print(f"{name} accuracy (with PCA): {accuracy_score(y_true=y_test, y_pred=predict)}")
end = timer()
print(f"Time elapsed: {end - start}")

SVC accuracy (with PCA): 0.9911111111111112
LogReg accuracy (with PCA): 0.9577777777777777
DecTree accuracy (with PCA): 0.8577777777777778
Time elapsed: 1.7567465830000004


In [5]:
# число компонент для 90% дисперсии 
pca_pipelines[0][1]['pca'].n_components_

21

In [6]:
# используем понижение размерности (TSNE)
# у TSNE из sk-learn нет метода transform, поэтому его 
# не получится использовать в пайплайне напрямую

start = timer()
tsne = TSNE(random_state=random_state)

X_processed = tsne.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, stratify=y)

tsne_pipelines = [
    ("SVC", Pipeline([("clf", SVC(random_state=random_state))])),
    ("LogReg", Pipeline([("clf", LogisticRegression(random_state=random_state, max_iter=log_reg_max_iter))])),
    ("DecTree", Pipeline([("clf", DecisionTreeClassifier(random_state=random_state))])),
]

for name, pipeline in tsne_pipelines:
    pipeline.fit(X_train, y_train)
    predict = pipeline.predict(X_test)
    print(f"{name} accuracy (with TSNE): {accuracy_score(y_true=y_test, y_pred=predict)}")

end = timer()
print(f"Time elapsed: {end - start}")
    

SVC accuracy (with TSNE): 0.9755555555555555
LogReg accuracy (with TSNE): 0.9288888888888889
DecTree accuracy (with TSNE): 0.9866666666666667
Time elapsed: 5.635096374999999


Выводы:
1. Использование PCA и TSNE привело к снижению accuracy для LogReg и DecisionTree; для PCA в меньшей степени, чем для TSNE
2. TSNE намного более вычислительно тяжелый метод, время выполнения отличается в 2 раза 